In [ ]:
using CMBLensing
using PythonPlot

: 

In [ ]:
Cℓ = camb(ℓmax=40000, r=0);

In [ ]:
μKarcminT = 1;
Nside = 128;
θpix = 0.5;
bandpass_mask = LowPass(10000);

In [ ]:
loglog(Cℓ.unlensed_scalar.TT)
loglog(Cℓ.lensed_scalar.TT)
loglog(noiseCℓs(μKarcminT=μKarcminT,ℓmax=40000).TT);

In [ ]:
(;f, f̃, ϕ, ds) = load_sim(
    θpix      = θpix,
    Nside     = Nside,
    T         = Float32,
    pol       = :I,
    μKarcminT = μKarcminT,
    L         = LenseFlow(10),
    seed      = 0,
    pixel_mask_kwargs = (edge_padding_deg=1, apodization_deg=0, num_ptsrcs=0),
    bandpass_mask     = bandpass_mask
);

In [ ]:
animate([f,f̃],fps=2)

In [ ]:
plot([f ϕ ds.d], title=[raw"true unlensed $T$" raw"true $\phi$" "data"])

In [ ]:
fJ,ϕJ, history = MAP_joint(ds, nsteps=20, progress=:summary);

In [ ]:
plot(getindex.(history, :logpdf))
xlabel("step")
ylabel("logpdf")

In [ ]:
plot(10^6 * [ϕJ ϕ], title=[raw"$\hat \phi_J$" raw"true $\phi$"], vlim=1.2)

In [ ]:
_,chain = sample_joint(ds, nsamps_per_chain=300, ϕstart=ϕJ, progress=:summary, symp_kwargs=[(N=25, ϵ=0.01)]);

In [ ]:
animate(getindex.(chain[1][2:5:end],:ϕ),fps=10,vlim=1.2e-6,title=raw"$\phi$ samples")